# Part II – Feedforward NNs, Autoencoders and Convolutional NNs

## This is a modification of previous notebook. Modifications: 

- Based on: `ML-ac-ficha6-PII-2.2.ipynb`

**Changes:**
    - Applying it for decision trees.

**Data import:**

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) =mnist.load_data()
# Normalize the images to the range [0, 1]
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
# Convert the labels to one-hot encoded vectors
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

**Neural Network:**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape

In [ ]:

# ENCODER #
# Define input layer
encoder_input = Input(shape=(28, 28, 1))
# Define the decoder layers
z = encoder_input
z = Flatten()(z)
z = Dense(64, activation='relu')(z)
z = Dense(10, activation='relu')(z)
encoder_layer = z 
encoder = Model(encoder_input, encoder_layer)

# DECODER #
## Input
decoder_input = Input(shape=encoder_layer.shape[1:])
##  Layers
y = decoder_input
y = Dense(64, activation='relu')(y)
y = Dense(784, activation='relu')(y)
y = Reshape((28, 28, 1))(y)
decoder_layer = y
## Model
decoder = Model(decoder_input, decoder_layer)

# AUTOENCODER #
## Model
autoencoder = Model(encoder_input, decoder(encoder(encoder_input)))
model = autoencoder
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_images, epochs=5, batch_size=64, validation_split=0.2)